In [44]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as sps
import math

from ipywidgets import interact

from ipywidgets import widgets
from tqdm.auto import tqdm

from dataclasses import dataclass
from typing import Union, Callable, Optional
from copy import deepcopy
from copy import error
from scipy.optimize import root_scalar, brentq
from dataclasses import dataclass
from scipy.special import iv
from scipy.stats import bernoulli
from scipy.interpolate import RectBivariateSpline
from scipy.optimize import newton
from scipy.stats import norm

import warnings
from scipy.stats import norm
warnings.filterwarnings("ignore")

from vol.vol import Heston

import hestonmc
from importlib import reload
reload(hestonmc)

from hestonmc import MarketState, HestonParameters, mc_price, simulate_heston_euler, simulate_heston_andersen_qe, simulate_heston_andersen_tg

## Broadie - Kaya scheme

In [8]:
def cir_chi_sq_sample(heston_params: HestonParameters,
                      dt:            float,
                      v_i:           np.array,
                      n_simulations: int):
    """Samples chi_squared statistics for v_{i+1} conditional on 
       v_i and parameters of Hestom model. 
        
    Args:
        heston_params (HestonParameters): parameters of Heston model
        dt (float): time step 
        v_i: current volatility value
        n_simulations (int): number of simulations.
        
    Returns:
        np.array: sampled chi_squared statistics 
    """
    kappa, vbar, gamma = heston_params.kappa, heston_params.vbar, heston_params.gamma
    
    barkappa=v_i*(4*kappa*np.exp(-kappa*dt))/(gamma**2 * (1 - np.exp(-kappa*dt)))
    d=(4*kappa*vbar)/(gamma**2)
    c=((gamma**2)/(4*kappa))*(1-np.exp(-kappa*dt))
    
    return  c*np.random.noncentral_chisquare(size = n_simulations, df   = d, nonc = barkappa)


def Phi(a:             Union[float, np.ndarray], 
        V:             Union[float, np.ndarray],
        time:          Union[float, np.ndarray],
        heston_params: HestonParameters
        ) -> np.ndarray:
    
    
    v0, rho, kappa, vbar, gamma = heston_params.v0, heston_params.rho, heston_params.kappa, \
                                        heston_params.vbar, heston_params.gamma
    dt = time[1::]-time[:-1:]
    
    A=np.array(a)
    gamma_a = np.sqrt(kappa**2 - 2*gamma**2*1j*A).reshape(1,1,len(A)).T
    
    E1 = np.exp(-kappa*dt)
    E2 = np.exp(-gamma_a * dt)
        
    P1 = ((1.0-E1)*gamma_a/(kappa*(1.0-E2)))*np.exp(-0.5*(gamma_a - kappa)*dt)
    
    P2_2 = kappa * (1.0 + E1)/(1.0 - E1) - gamma_a*(1.0+E2)/(1-E2)
    P2 = np.exp( (V[:,1::]+V[:,:-1:])/(gamma_a**2) * P2_2 )
    
    P3_1 = np.sqrt(V[:,1::]*V[:,:-1:])*4*gamma_a * np.exp(-0.5*gamma_a*dt) /(gamma**2 * (1.0 - E2))
    P3_2 = np.sqrt(V[:,1::]*V[:,:-1:])*4*kappa*np.exp(-0.5*kappa*dt)/(gamma**2 * (1 - E1))
    d=(4*kappa*vbar)/(gamma**2)
    P3 = iv(0.5*d - 1, P3_1)/iv(0.5*d - 1, P3_2)
    
    return P1*P2*P3

def Pr(V:             np.ndarray, 
       time:          np.ndarray,
       X:             Union[np.ndarray, float],
       heston_params: HestonParameters,
       h:             float=1e-2, 
       eps:           float=1e-2
       ) -> np.ndarray:
    
    x=np.array(X)
    P=h*x/np.pi
    S = 0.0
    j = 1
    while(True):
        Sin=np.sin(h*j*x)/j
        Phi_hj=Phi(h*j, V, time, heston_params)
        S+=Sin.reshape(1,1,len(x)).T * Phi_hj[0]
        if np.all(Phi_hj[0]<np.pi*eps*j/2.0):
            break
        j=j+1
    
    S=S*2.0/np.pi
    return P+S

def IV(V:             np.ndarray, 
       time:          np.ndarray,
       heston_params: HestonParameters,
       h:             float=1e-2, 
       eps:           float=1e-2
       ) -> np.ndarray:
    
    U=np.random.uniform(size=(V.shape[0], V.shape[1] - 1))
    
    def f(x,i,j):
        P=Pr(V, time, x, heston_params, h, eps)
        return (P-U)[0][i,j]
    
    IVar = np.zeros((V.shape[0], V.shape[1] - 1))
    
    for i in range(IVar.shape[0]):
        for j in range(IVar.shape[1]):     
            IVar[i,j]=root_scalar(f, args=(i,j), x0=0.5, method='newton')
    return IVar

## Tests

In [45]:
heston_parameters = HestonParameters(kappa = 1.3125, gamma = 0.5125, rho = -0.3937, vbar = 0.0641, v0 = 0.3)
# kappa = 1.3125, gamma = 0.7125, rho = -0.3937, vbar = 0.0641, v0 = 0.3
state = MarketState(stock_price = 100., interest_rate = 0.2)

params = heston_parameters
model = Heston(state.stock_price, heston_parameters.v0, heston_parameters.kappa, heston_parameters.vbar, heston_parameters.gamma, heston_parameters.rho, state.interest_rate)
kwargs = {}

In [46]:
def get_payoff(maturity: float,
               strike: float,
               interest_rate: float = 0.):
    def payoff(St: np.ndarray):
        DF = np.exp( - interest_rate * maturity)
        return np.maximum(St - strike, 0.)*DF

    return payoff

### At the money

In [47]:
strike = 100.
T = 1.
payoff = get_payoff(T, strike, state.interest_rate)

In [48]:
theory = model.call_price(T, strike)
print(theory)

22.735939055676866


In [ ]:
# np.random.seed(42)
euler    = mc_price(dt = 5e-2, absolute_error=5e-2, market_state = state, payoff = payoff, simulate = simulate_heston_euler,
                    debug = False, params = params, T = T, **kwargs)
print(euler)

In [20]:
# np.random.seed(42)
andersen = mc_price(dt = 5e-2, absolute_error=5e-2, market_state = state, payoff = payoff, simulate = simulate_heston_andersen_qe, params = params, T = T, **kwargs)
print(andersen)

Number of iterations:   646
Number of simulations:  6460000

25.54671704399664


In [22]:
r_x = np.load(r"Data\anderson tg\r_x.npy")
f_nu_y = np.load(r"Data\anderson tg\f_nu_y.npy")
f_sigma_y = np.load(r"Data\anderson tg\f_sigma_y.npy")

In [23]:
#np.random.seed(42)
kwargs = {'x_grid' : r_x, 'f_nu_grid' : f_nu_y, 'f_sigma_grid' : f_sigma_y }
andersen = mc_price(dt = 1e-2, absolute_error=5e-4, market_state = state, payoff = payoff, simulate = simulate_heston_andersen_tg, params = params, T = T,debug=True, **kwargs)
print(andersen)

Current price: 25.6948 +/- 1.3337
Current price: 26.1367 +/- 0.9497
Current price: 26.1767 +/- 0.7812
Current price: 26.1023 +/- 0.6758
Current price: 26.2121 +/- 0.6056
Current price: 26.2809 +/- 0.5539
Current price: 26.2666 +/- 0.5123
Current price: 26.1937 +/- 0.4778
Current price: 26.1900 +/- 0.4501
Current price: 26.2193 +/- 0.4274
Current price: 26.2171 +/- 0.4072
Current price: 26.2633 +/- 0.3902
Current price: 26.2859 +/- 0.3753
Current price: 26.3137 +/- 0.3616
Current price: 26.3315 +/- 0.3492
Current price: 26.3228 +/- 0.3380
Current price: 26.3128 +/- 0.3281
Current price: 26.3118 +/- 0.3190
Current price: 26.3094 +/- 0.3106
Current price: 26.2882 +/- 0.3027
Current price: 26.3349 +/- 0.2960
Current price: 26.3406 +/- 0.2894
Current price: 26.3367 +/- 0.2829
Current price: 26.3342 +/- 0.2769
Current price: 26.3267 +/- 0.2711
Current price: 26.3282 +/- 0.2658
Current price: 26.3233 +/- 0.2609
Current price: 26.3193 +/- 0.2562
Current price: 26.3242 +/- 0.2518
Current price:

KeyboardInterrupt: 

In [35]:
for i in range(5):
    euler    = mc_price(dt = 1e-2, absolute_error=5e-2, market_state = state, payoff = payoff, simulate = simulate_heston_euler, params = params, T = T, **kwargs)
    print(euler)
print("\n\n")
for i in range(5):
    euler    = mc_price(dt = 1e-2, absolute_error=5e-2, market_state = state, payoff = payoff, simulate = simulate_heston_andersen_qe, params = params, T = T, **kwargs)
    print(euler)

Number of iterations:   499
16.141856095186746
Number of iterations:   500
16.16485690142346
Number of iterations:   501
16.162006575869245
Number of iterations:   501
16.172163674801727
Number of iterations:   502
16.16445622313361



Number of iterations:   497
16.13599096813387
Number of iterations:   495
16.138183069198718
Number of iterations:   497
16.144927377340313
Number of iterations:   494
16.13851449693267
Number of iterations:   497
16.14301732726582


### Out of the money

In [53]:
strike = 500.
payoff = get_payoff(T, strike, state.interest_rate)

In [54]:
model.call_price(T, strike)

0.0017623741249792602

In [55]:
np.random.seed(42)
euler    = mc_price(dt = 1e-2, market_state = state, payoff = payoff, simulate = simulate_heston_euler, params = params, T = T, **kwargs)
print(euler)

Number of iterations:   1
Number of simulations:  10000

0.0


In [56]:
np.random.seed(42)
andersen = mc_price(dt = 1e-2, market_state = state, payoff = payoff, simulate = simulate_heston_andersen_qe, params = params, T = T, **kwargs)
print(andersen)

Number of iterations:   1
Number of simulations:  10000

0.0023274136704930243


### Implied volatility

In [ ]:
def d_1(q,T,S,K,r):
    denom=1/(q*np.sqrt(T))
    log=np.log(S/K)
    s2=T*(r+q*q/2)
    return denom*(log+s2)

def d_2(q,T,S,K,r):
    denom=1/(q*np.sqrt(T))
    log=np.log(S/K)
    s2=T*(r-q*q/2)
    
    return denom*(log+s2)

def calc_iv(option: CallStockOption, state: MarketState, option_price: float):
    
    T=option.expiration_time
    K=option.strike_price
    S=state.stock_price
    r=state.interest_rate    
    N=sps.norm()
    
    
    def f(q):
        d1=d_1(q,T,S,K,r)
        d2=d_2(q,T,S,K,r)
        return S*N.cdf(d1)-K*(np.exp(-r*T))*N.cdf(d2)-option_price
    
    def fprime(q):
        d1=d_1(q,T,S,K,r)
        return S*sps.norm().pdf(d1)*np.sqrt(T)
    
    sol = root_scalar(f, x0=0.5, fprime=fprime, method='newton')
    return sol.root



In [ ]:
strikes = np.arange(80, 120, 2)
strikes

In [ ]:
call_price = np.array([30.096826945107107,
 28.86569299569351,
 27.627749990421822,
 26.468755491540694,
 25.31876428557471,
 24.24782290614315,
 23.16942316862601,
 22.12935496069162,
 21.079816460198398,
 20.20119494244462,
 19.288858679686502,
 18.392107111749084,
 17.50127598061397,
 16.701957005015792,
 15.90122188123928,
 15.14196593075513,
 14.43621999411162,
 13.722786552930495,
 13.069519364876335,
 12.47747576375882])

In [ ]:
IV = np.empty_like(call_price)

for j in range(len(call_price)):
        IV[j] = calc_iv(option=CallStockOption(strikes[j], T), 
                           state=state, 
                           option_price=call_price[j])

In [ ]:
_, ax = plt.subplots(figsize=(15, 5))

ax.plot(strikes, IV, "o-")
# ax.legend()
ax.set_xlabel("Strike, $")
ax.set_ylabel("IV")
ax.set_title("Implied Volatility")
plt.show()

In [ ]:
model = Heston(state.stock_price, heston_parameters.v0, heston_parameters.kappa, 
                         heston_parameters.vbar, heston_parameters.gamma, heston_parameters.rho, state.interest_rate)

In [ ]:
call_price = np.zeros(20)

for j in range(20):
    call_price[j] = model.call_price(2., strikes[j])

In [ ]:
IV = np.empty_like(call_price)

for j in range(len(call_price)):
        IV[j] = calc_iv(option=CallStockOption(strikes[j], T), 
                           state=state, 
                           option_price=call_price[j])

In [ ]:
_, ax = plt.subplots(figsize=(15, 5))

ax.plot(strikes, IV, "o-")
# ax.legend()
ax.set_xlabel("Strike, $")
ax.set_ylabel("IV")
ax.set_title("Implied Volatility")
plt.show()

In [ ]:
Z = np.random.normal(size=1000)